In [10]:
import utils
import numpy as np

from tqdm import tqdm
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [11]:
# dji = utils.get_dji()
spy = utils.get_spy()
# qqq = utils.get_qqq()

symbols = set(spy['Symbol'])
# symbols = set(dji['Symbol']) | set(spy['Symbol']) | set(qqq['Symbol'])

SPY: Contains 503 tickers.


In [12]:
args = utils.load_args()
startdate, enddate = utils.get_dates(args)

market_return, riskfree_return, nonholiday_dates = utils.get_benchmark_data(args.benchmark.market, args.benchmark.riskfree, startdate, enddate)
market_data = utils.get_pdr_data(args.benchmark.market, startdate-utils.parse_period_date(args.sharpe.period), enddate, progress=False, clean=False)
stocks_data = utils.get_pdr_data(symbols, startdate-utils.parse_period_date(args.sharpe.period), enddate, progress=True, clean=True)

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-09-17 -> 2024-03-17)')


In [13]:
sharpe_update_dates = utils.linspace_datetime(nonholiday_dates, startdate, enddate, delta=utils.parse_period_date(args.sharpe.period))
rebalance_update_dates = utils.linspace_datetime(nonholiday_dates, startdate, enddate, delta=utils.parse_period_date(args.rebalance.period))

In [14]:
portfolio = utils.Portfolio()
stocks_return = utils.compute_log_return(stocks_data, was_annual=False, retain_symbols=True)

X = []
X_market = []

for dt in tqdm(nonholiday_dates, desc="nonholiday_dates"):
    if dt in rebalance_update_dates:
        # Compute sharpe.
        startdate_temp = dt - utils.parse_period_date(args.sharpe.period)
        enddate_temp = dt
        sharpe_df = utils.get_stocks_sharpe_from_data(stocks_data, args.benchmark.riskfree, startdate_temp, enddate_temp)
        
        # Rebalance portfolio.
        market_sharpe = utils.get_stocks_sharpe_from_data(market_data, args.benchmark.riskfree, startdate_temp, enddate_temp)
        portfolio.rebalance(sharpe_df.head(args.params.no_of_companies), min_threshold=market_sharpe[0])

        if args.display.verbose:
            print("Date:", dt)
            print("Market Sharpe:", market_sharpe[0])
            portfolio.display()
            print('-'*80)
    
    # Update portfolio with today's returns.
    portfolio.update(stocks_return, dt)
    X.append(portfolio.value)
    X_market.append(market_return.loc[dt.strftime('%Y-%m-%d')])

nonholiday_dates:   7%|▋         | 83/1258 [00:00<02:03,  9.52it/s]

Date: 2019-03-18 00:00:00
Market Sharpe: -0.023632414682977096
Ticker	Utility	Weight
AMT:	0.1616	0.2153
ENPH:	0.1614	0.2151
AES:	0.1507	0.2009
O:	0.146	0.1945
SBUX:	0.1306	0.1741
--------------------------------------------------------------------------------


nonholiday_dates: 100%|██████████| 1258/1258 [00:01<00:00, 773.13it/s]


In [15]:
rebalance_update_dates

[datetime.datetime(2019, 3, 18, 0, 0)]

In [16]:
portfolio.display()

Ticker	Utility	Weight
AMT:	0.1616	0.2153
ENPH:	0.1614	0.2151
AES:	0.1507	0.2009
O:	0.146	0.1945
SBUX:	0.1306	0.1741


In [26]:
print("Dates:           ", nonholiday_dates[0].date(), "to", nonholiday_dates[-1].date())
print("Portfolio return:", portfolio.value)
print("Market return:   ", sum(X_market)[0])

Dates:            2019-03-18 to 2024-03-14
Portfolio return: 0.5405892915301703
Market return:    0.5987868798321898


In [33]:
real = np.array([1.91, 1072, -16.89, -25, +25])
real.dot(list(portfolio.current_tickers.values()))

170.4424214686176

In [18]:
# Couple of hings to try:
# - Check why the market return does not match actual performance.
# - Try Sortino's ratio instead.